In [1]:
! pip install selenium
! pip install beautifulsoup4
! pip install pandas
! pip install openpyxl
! pip install webdriver-manager
# ! pip install selenium


[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [75]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

In [103]:
def login_linkedin(driver, username, password):
    username = "kumarashwani1208@gmail.com"
    password = "Dayachand@7037"
    driver = webdriver.Chrome()  # Replace with the appropriate driver for your browser

    driver.get("https://www.linkedin.com/login")
    driver.find_element(By.ID, "username").send_keys(username)
    driver.find_element(By.ID, "password").send_keys(password)
    driver.find_element(By.XPATH, "//button[@type='submit']").click()
    time.sleep(2)  # Add a pause before locating elements

login_linkedin(driver,username,password)

In [90]:
def search_profiles(driver, first_name, last_name):
    search_url = "https://www.linkedin.com/search/results/people/"
    driver.get(search_url)
    
    # Enter the search term in the search box
    search_box = driver.find_element(By.XPATH, "//input[@placeholder='Search']")
    search_box.send_keys(f"{first_name} {last_name}")
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)  # Wait for search results to load

    # Scroll down to load more results if necessary
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    profiles = []

    # Find profile cards in search results
    profile_cards = soup.find_all('div', class_="entity-result__content")[:10]  # Get top 10 results
    for profile in profile_cards:
        try:
            name = profile.find('span', class_="entity-result__title-text").get_text(strip=True)
            occupation = profile.find('div', class_="entity-result__primary-subtitle").get_text(strip=True)
            location = profile.find('div', class_="entity-result__secondary-subtitle").get_text(strip=True)
            link = profile.find('a', href=True)['href']
            
            profiles.append({
                "name": name,
                "occupation": occupation,
                "location": location,
                "link": f"https://www.linkedin.com{link}"
            })
        except AttributeError:
            continue  # Skip profiles with missing info

    return profiles


In [101]:
import csv
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Function to save profiles to CSV
def save_to_csv(profiles, output_file):
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=["Name", "Title", "Profile Link"])
        writer.writeheader()
        for profile in profiles:
            writer.writerow(profile)

# Function to search and extract profiles
def search_profiles(driver, first_name, last_name, output_file):
    search_url = "https://www.linkedin.com/search/results/people/"
    driver.get(search_url)

    try:
        # Wait for the search box to load
        search_box = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder*='Search']"))
        )
        search_box.send_keys(f"{first_name} {last_name}")
        search_box.send_keys(Keys.RETURN)
        
        # Wait for search results to load
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "search-results-container"))
        )
        
        print("Search completed successfully.")
        
        # Extract profile data
        profiles = []
        
        # Try to find the individual profile elements using the correct selectors
        profile_elements = driver.find_elements(By.CSS_SELECTOR, ".entity-result")
        
        for profile in profile_elements:
            try:
                # Extract Name, Title, and Profile Link
                name = profile.find_element(By.CSS_SELECTOR, ".entity-result__title-text").text
                title = profile.find_element(By.CSS_SELECTOR, ".entity-result__primary-subtitle").text
                link = profile.find_element(By.CSS_SELECTOR, "a.app-aware-link").get_attribute("href")
                
                profiles.append({"Name": name, "Title": title, "Profile Link": link})
            except Exception as e:
                print(f"Error extracting data from profile: {e}")
                continue
        
        # Save profiles to a CSV file
        output_file = f"linkedin_profiles_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        save_to_csv(profiles, output_file)
        
        print(f"Profiles saved to {output_file}")
    
    except TimeoutException:
        print("Search bar or results did not load in time.")
        return None

# Main script
if __name__ == "__main__":
    # LinkedIn credentials
    username = "kumarashwani1208@gmail.com"
    password = "Dayachand@7037"
    first_name = input("Enter First Name: ")
    last_name = input("Enter Last Name: ")

    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.headless = False  # Set to True for headless mode
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()

    try:
        if login_linkedin(driver, username, password):
            search_profiles(driver, first_name, last_name, "linkedin_profiles.csv")
        else:
            print("Login was not successful.")
    
    finally:
        driver.quit()


Login was not successful.


In [86]:
# Main
username = "kumarashwani1208@gmail.com"
password = "Dayachand@7037"
first_name = input("Enter First Name: ")
last_name = input("Enter Last Name: ")
driver = webdriver.Chrome()  # Replace with the appropriate driver for your browser
try:
    login_linkedin(driver, username, password)
    profiles = search_profiles(driver, first_name, last_name)
    save_to_csv(profiles)
finally:
    driver.quit()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//input[@placeholder='Search']"}
  (Session info: chrome=130.0.6723.117); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF791EE38A5+3004357]
	(No symbol) [0x00007FF791B79970]
	(No symbol) [0x00007FF791A2582A]
	(No symbol) [0x00007FF791A75B8E]
	(No symbol) [0x00007FF791A75E7C]
	(No symbol) [0x00007FF791ABEC27]
	(No symbol) [0x00007FF791A9BC1F]
	(No symbol) [0x00007FF791ABBA4C]
	(No symbol) [0x00007FF791A9B983]
	(No symbol) [0x00007FF791A67628]
	(No symbol) [0x00007FF791A68791]
	GetHandleVerifier [0x00007FF791F0A00D+3161901]
	GetHandleVerifier [0x00007FF791F5E060+3506048]
	GetHandleVerifier [0x00007FF791F5400D+3465005]
	GetHandleVerifier [0x00007FF791CD0EEB+830987]
	(No symbol) [0x00007FF791B8467F]
	(No symbol) [0x00007FF791B809D4]
	(No symbol) [0x

In [78]:
def save_to_csv(profiles):
    with open('linkedin_profiles6.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Name", "Occupation", "Location", "Profile Link"])
        for profile in profiles:
            writer.writerow([profile['name'], profile['occupation'], profile['location'], profile['link']])

# Main

In [84]:
# login_linkedin(driver, username, password)
driver = webdriver.Chrome()  # Replace with the appropriate driver for your browser
try:
    login_linkedin(driver, username, password)
    # profiles = search_profiles(driver, first_name, last_name)
    # save_to_csv(profiles)
finally:
    driver.quit()